# Ćwiczenia 6. Walidacja Krzyżowa

## PyTorch na następne ćwiczenia.

Proszę zainstalować pakiet [PyTorch](https://pytorch.org/) oraz torchvision na kolejne zajęcia. Jeśli używane, mając swoje środowisko aktywne użyć:

 * GPU: `conda install pytorch torchvision cudatoolkit=9.0 -c pytorch`
 * tylko CPU: `conda install pytorch torchvision cpuonly  -c pytorch`

## Klasyfikacja

Dzisiaj na zajęciach zajmiemy się problemem klasyfikacji. Podobnie do regresji liniowej jest to przykład uczenia nadzorowanego, ale zamiast przewidywać konkretną liczbę dla danej obserwacji, przewidujemy jego przynajeżność do jednej z *k* klas. Na tych zajęciach będziemy rozważać klasyfikacje binarną, czyli uczyć modele odpowiadające funkcji:

$$ f(x) = y, \quad y \in \{0,1\} $$

Poniżej ładowane są dane, do razu już podzielone na dwie części.

In [ ]:
import numpy as np
from utils import get_data

X_train, X_test, y_train, y_test = get_data()

## Zadanie 1.1 (0.5 pkt.)

Używając modelu [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) z pakietu sklearn uzyskać 100% dokładność (mierzoną miarą [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))na zbiorze treningowym. Państwa zadanie polega na dobraniu parametru `gamma`, dla ułatwienia proszę nie zmieniać pozostałych domyślnych parametrów. Zalecany przedział parametru `gamma` to wartości z przedziału [0, 1] w skali logarytmicznej.

In [ ]:
from sklearn.svm import SVC

# your code here


In [ ]:
# test
best_gamma = best_gamma # ???

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
train_acc = svm.score(X_train, y_train)

assert train_acc == 1.

## Zadanie 1.2 (0.5 pkt.)
Używając tej samej rodziny modeli znajdź tym razem taką wartość `gamma`, która daje najlepszy wynik na zbiorze **testowym**. Powinieneś(-aś) być w stanie osiągnąć wynik co najmniej `0.95` accuracy. 

In [ ]:
from sklearn.svm import SVC

# your code here


In [ ]:
# test
best_gamma = best_gamma # ???

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
test_acc = svm.score(X_test, y_test)

assert test_acc >= 0.95

### Problem.

**W poprzednim zadaniu zakładaliśmy, że podział na zbiór trenujący/testujący jest nam podany z góry, ale co jeśli go nie mamy?**

Możemy oczywiście sami arbitralnie wybrać część datasetu i uznać ją za nasz zbiór testowy, ale to mogą się z tym wiązać dodatkowe problemy: co jeśli wybrany przez nas fragment jest akurat różny od reszty datasetu, lub odwrotnie?

**Rozwiązanie:** Walidacja Krzyżowa.

1. Podziel dataset na zadaną przez parametr `k` liczbę (prawie) równych grup.
2. Dla każdego podziału, zwróć jedną z tych części jako zbiór testowy, a sumę reszty jako zbiór treningowy.
3. Po nauczeniu łącznie `k` modeli, uśrednij ich wyniki na zbiorach testowych i zwróć jako ostateczny wynik.

## Zadanie 2. (2 pkt.)

Państwa zadaniem jest zaimplementowanie walidacji krzyżowej, czyli funkcji, która dla podanego datasetu w postaci macierzy danych `X` i wektora etykiet `y` zwróci listę `k` krotek: 
  
  `(treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)`
  
podziałów dokonanych przez walidację krzyżową. Następnie należy użyć modelu z poprzedniego zadania dla policzenia dokładności na zbiorze testowym dla walidacji krzyżowej.

Proszę **nie** korzystać z gotowych rozwiązań dostępnych w pakiecie sklearn.


In [ ]:
from typing import List, Tuple


def cross_validation(X: np.ndarray, y: np.ndarray, k: int) -> List[Tuple[np.ndarray, np.ndarray, 
                                                                         np.ndarray, np.ndarray]]:
    
    # your code here
    
    return ???

In [ ]:
from checker import test_cv

test_cv(cross_validation)

In [ ]:
X, y = get_data(split=False)

# your code here
    
cv_accuracy = ???

## Zadanie 3 (1 pkt.)

Mając już lepszą metodę walidacji naszego rozwiązania Państwa zadaniem jest znalezienia najlepszego zestawu hiperparametrów dla modelu z poprzednich zadań, lecz tym razem będziemy szukać również parametru `C`. Parametru C zaleca się szukać w przedziale $(0, + \infty)$ również w skali logarytmicznej.

W zadaniu należy oczywiście skorzystać z zaimplementowanej przez Państwa walidacji krzyżowej. Ponieważ dla dwóch parametrów `C` oraz `gamma` możliwych kombinacji do przetestowania robi są dość sporo dla przetestowania dużych zakresów zalecane są również inne metody przeszukiwania, takie jak:

* przeszukiwanie po kolei z jednym z parametrów ustalonym na stałą.
* przeszukiwanie losowe obu parametrów

Oczywiście jeśli zasoby na to pozwalają można szukać tzw. "grid searchem".

Powinno udać się Państwu wyciągnąć przynajmniej `0.94` accuracy na walidacji krzyżowej.

In [ ]:
from sklearn.svm import SVC

X, y = get_data(split=False)

# your code here


## Zadanie 4. (3 punkty)

Załóżmy, że naszym problemem jest zdecydowanie, która rodzina modeli *SVM* najlepiej radzi sobei z naszym datasetem. Przez rodzinę rozumiemy tutaj modele SVM z różną *funkcją bazwoą* (zwaną często *funkcją jądra*). W pakiecie mamy dostępne kilka możliwości, włącznie z podawaniem swoich własnych, ale w tym zadaniu skupimy się na czterech dostępnych od ręki: `linear`, `poly`, `rbf`, `sigmoid`.

Wiemy jak znaleźć najlepszy zestaw parametrów dla danej rodziny modeli, zrobiliśmy to do tej pory dla domyślnej funkcji bazowej czyli `rbf`. Jak jednak mamy "uczciwie" porównać wyniki modeli pomiędzy sobą? Do tej pory patrzyliśmy na wyniki modelu dla datasetu testowego i to na podstawie tego wyniku wybieraliśmy najlepsze hiperparametry. Jakie mogą być z tym problemy? Overfitting?

Rozwiązanie: jeszcze jedna walidacja krzyżowa!

1. Pierwsza walidacja krzyżowa podzieli nam nasz zbiór na treningowy i testowy. Te testowe zbiory będą naszymi ostatecznymi zbiorami testowymi, na których nie będziemy w żaden sposób się uczyć czy szukać hiperparametrów. 
2. Następnie nasz zbiór treningowy podzielimy ponownie walidacją krzyżową na dwie części: faktyczny treningowy i walidacyjny. Tych dwóch podziałów będziemy używać jak poprzednio do uczenia modelu i testowania hiperparametrów.
3. Po znalezieniu najlepszego zestawu hiperparametrów nauczymy ostatecznie nasz model na sumie zbiorów treningowego i walidacyjnego i sprawdzimy jego dokładność na ostatecznym zbiorze testowym.


**Uwaga**: parametr `C` używany jest dla każdej możliwej funkcji bazowej. Proszę sprawdzić jakie parametry są używane dla jakich funkcji jądra. 
**Hint**: parametry, które mogą państwa interesować to oczywiście `kernel`, oraz `C`, `degree`, `gamma`, `coef0`.

In [ ]:
from sklearn.svm import SVC

X, y = get_data(split=False)

# yout code here
